# Simple sensitivity study

Author: Raghav Kansal

In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import mplhep as hep
from matplotlib import colors

from boostedhh import utils, hh_vars, plotting
from boostedhh.utils import PAD_VAL
from bbtautau import bbtautau_vars

import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("boostedhh.utils")
logger.setLevel(logging.DEBUG)

In [2]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

In [3]:
MAIN_DIR = Path("../../../")
CHANNEL = "electron"  # options: "hadronic", "electron", "muon"

plot_dir = MAIN_DIR / f"plots/SensitivityStudy/25Jan22{CHANNEL}"
plot_dir.mkdir(parents=True, exist_ok=True)

year = "2022"
signal_samples_tag = "24Nov21ParTMass_v12_private_signal"
data_samples_tag = "24Nov21ParTMass_v12_private_signal"

## Define and load samples

In [14]:
base_dir = Path("/ceph/cms/store/user/rkansal/bbtautau/skimmer/")

# define samples to load
samples = {
    "jetmet": utils.Sample(
        path=base_dir / data_samples_tag,
        selector="JetHT|JetMET",
        label="JetMET",
        isData=True,
        year=year,
    ),
    "tau": utils.Sample(
        path=base_dir / data_samples_tag,
        selector="Tau_Run",
        label="Tau",
        isData=True,
        year=year,
    ),
    "muon": utils.Sample(
        path=base_dir / data_samples_tag,
        selector="Muon_Run",
        label="Muon",
        isData=True,
        year=year,
    ),
    "egamma": utils.Sample(
        path=base_dir / data_samples_tag,
        selector="EGamma_Run",
        label="EGamma",
        isData=True,
        year=year,
    ),
    "bbtt": utils.Sample(
        path=base_dir / signal_samples_tag,
        selector=hh_vars.bbtt_sigs["bbtt"],
        label=r"HHbb$\tau\tau$",
        isData=False,
        year=year,
    ),
}

# pick signal key based on channel
SIG_KEYS = {"hadronic": "bbtthh", "electron": "bbtthe", "muon": "bbtthmu"}
SIG_KEY = SIG_KEYS[CHANNEL]

# pick relevant data samples based on channel
DATA_KEYS = {
    "hadronic": ["jetmet", "tau"],
    "electron": ["jetmet", "tau", "egamma"],
    "muon": ["jetmet", "tau", "muon"],
}[CHANNEL]

# pick relevant lepton dataset based on channel
LEPTON_DATASET = {"hadronic": None, "electron": "egamma", "muon": "muon"}[CHANNEL]

LEPTON_TRIGGERS = {
    "hadronic": None,
    "electron": bbtautau_vars.HLT_he,
    "muon": bbtautau_vars.HLT_hmu,
}[CHANNEL]

for key in ["jetmet", "tau", "egamma", "muon"]:
    if key not in DATA_KEYS:
        del samples[key]

In [ ]:
# pt_cut = 250
# msd_cut = 40

filters = [
    [
        ("('ak8FatJetPt', '0')", ">=", 250),
        ("('ak8FatJetPNetmassLegacy', '0')", ">=", 50),
        ("('ak8FatJetPt', '1')", ">=", 200),
        # ("('ak8FatJetMsd', '0')", ">=", msd_cut),
        # ("('ak8FatJetMsd', '1')", ">=", msd_cut),
        # ("('ak8FatJetPNetXbb', '0')", ">=", 0.8),
    ],
]

# dictionary that will contain all information (from all samples)
events_dict = {}
for key, sample in samples.items():
    events_dict[key] = utils.load_sample(sample, filters)

events_dict["bbtthh"] = events_dict["bbtt"][events_dict["bbtt"]["GenTauhh"][0]]
events_dict["bbtthmu"] = events_dict["bbtt"][events_dict["bbtt"]["GenTauhmu"][0]]
events_dict["bbtthe"] = events_dict["bbtt"][events_dict["bbtt"]["GenTauhe"][0]]
del events_dict["bbtt"]

cutflow = pd.DataFrame(index=list(events_dict.keys()))
utils.add_to_cutflow(events_dict, "Preselection", "finalWeight", cutflow)
cutflow

## Triggers

In [ ]:
for skey in SIG_KEYS.values():
    triggered = np.sum([events_dict[skey][hlt][0] for hlt in bbtautau_vars.HLT_hmu], axis=0).astype(
        bool
    )
    events_dict[skey] = events_dict[skey][triggered]

### Data (overlap removal)

In [15]:
trigdict = {"jetmet": {}, "tau": {}}

if LEPTON_DATASET:
    trigdict[LEPTON_DATASET] = {}

for key, d in trigdict.items():
    d["all"] = np.sum([events_dict[key][hlt][0] for hlt in bbtautau_vars.HLT_hmu], axis=0).astype(
        bool
    )
    d["jets"] = np.sum(
        [
            events_dict[key][hlt][0]
            for hlt in bbtautau_vars.HLT_dict["PNet"] + bbtautau_vars.HLT_dict["PFJet"]
        ],
        axis=0,
    ).astype(bool)
    d["taus"] = np.sum([events_dict[key][hlt][0] for hlt in bbtautau_vars.HLT_taus], axis=0).astype(
        bool
    )

    d["taunojets"] = ~d["jets"] & d["taus"]

    if LEPTON_DATASET:
        d[LEPTON_DATASET] = np.sum(
            [events_dict[key][hlt][0] for hlt in LEPTON_TRIGGERS], axis=0
        ).astype(bool)

        d[f"{LEPTON_DATASET}noothers"] = ~d["jets"] & ~d["taus"] & d[LEPTON_DATASET]

Checking event loss by flipping triggers (can skip)

In [ ]:
# xor = np.setdiff1d(
#     events_dict["jetmet"][trigdict["jetmet"]["nojettau"]]["event"][0],
#     events_dict["tau"][trigdict["tau"]["nojettau"]]["event"][0],
# )

# print(len(xor) / len(events_dict["jetmet"]))

# xor = np.setdiff1d(
#     events_dict["tau"][trigdict["tau"]["jetnotau"]]["event"][0],
#     events_dict["jetmet"][trigdict["jetmet"]["jets"]]["event"][0],
# )

# print(len(xor) / len(events_dict["tau"]))

Apply overlap removal

In [17]:
events_dict["jetmet"] = events_dict["jetmet"][trigdict["jetmet"]["jets"]]
events_dict["tau"] = events_dict["tau"][trigdict["tau"]["taunojets"]]
events_dict[LEPTON_DATASET] = events_dict[LEPTON_DATASET][
    trigdict[LEPTON_DATASET][f"{LEPTON_DATASET}noothers"]
]

In [ ]:
utils.add_to_cutflow(events_dict, "Triggers", "finalWeight", cutflow)
cutflow

## FatJet Gen Matching

In [19]:
events = events_dict[SIG_KEY]
higgs = utils.make_vector(events, "GenHiggs")
bb = utils.make_vector(events, "Genbb")
tt = utils.make_vector(events, "GenTau")
fatjets = utils.make_vector(events, "ak8FatJet", mstring="Msd")

In [20]:
minbb = np.min(higgs[:, 0:1].deltaR(bb), axis=1)
mintau = np.min(higgs[:, 0:1].deltaR(tt), axis=1)
genhbb1 = minbb < mintau

# minbb = np.min(higgs[:, 1:2].deltaR(bb), axis=1)
# mintau = np.min(higgs[:, 1:2].deltaR(tt), axis=1)
# genhbb2 = minbb < mintau  # overlap with genhb1 < 0.5% of the time

genhbb_mask = np.vstack([genhbb1, ~genhbb1]).T
genhbb = higgs[genhbb_mask]
genhtt = higgs[~genhbb_mask]

In [21]:
fjbbdr = fatjets.deltaR(genhbb[:, np.newaxis])
fjidbb = np.argmin(fjbbdr, axis=1)
fjttdr = fatjets.deltaR(genhtt[:, np.newaxis])
fjidtt = np.argmin(fjttdr, axis=1)
# 5% of events have overlap out of which only 5% actually have two jets both close to a gen Higgs,
# so ignoring these overlap events for now
overlap = fjidbb == fjidtt

In [ ]:
np.mean(overlap)

## Taggers

In [23]:
taggers_dict = {}
taukey = {"hadronic": "Xtauhtauh", "electron": "Xtauhtaue", "muon": "Xtauhtaum"}[CHANNEL]

for key, events in events_dict.items():
    tvars = {}

    qcdouts = ["QCD0HF", "QCD1HF", "QCD2HF"]
    topouts = ["TopW", "TopbW", "TopbWev", "TopbWmv", "TopbWtauhv", "TopbWq", "TopbWqq"]
    tvars["PQCD"] = sum([events[f"ak8FatJetParT{key}"] for key in qcdouts]).to_numpy()
    tvars["PTop"] = sum([events[f"ak8FatJetParT{key}"] for key in topouts]).to_numpy()

    for disc in ["Xbb", taukey]:
        tvars[f"{disc}vsQCD"] = np.nan_to_num(
            events[f"ak8FatJetParT{disc}"] / (events[f"ak8FatJetParT{disc}"] + tvars["PQCD"]),
            nan=PAD_VAL,
        )
        tvars[f"{disc}vsQCDTop"] = np.nan_to_num(
            events[f"ak8FatJetParT{disc}"]
            / (events[f"ak8FatJetParT{disc}"] + tvars["PQCD"] + tvars["PTop"]),
            nan=PAD_VAL,
        )

        # make sure not to choose padded jets below by accident
        nojet3 = events["ak8FatJetPt"][2] == PAD_VAL
        tvars[f"{disc}vsQCD"][:, 2][nojet3] = PAD_VAL
        tvars[f"{disc}vsQCDTop"][:, 2][nojet3] = PAD_VAL

    tvars["PNetXbbvsQCD"] = np.nan_to_num(
        events["ak8FatJetPNetXbbLegacy"]
        / (events["ak8FatJetPNetXbbLegacy"] + events["ak8FatJetPNetQCDLegacy"]),
        nan=PAD_VAL,
    )

    # jet assignment
    fjbbpick = np.argmax(tvars["XbbvsQCD"], axis=1)
    fjttpick = np.argmax(tvars[f"{taukey}vsQCD"], axis=1)
    overlap = fjbbpick == fjttpick
    fjbbpick[overlap] = np.argsort(tvars["XbbvsQCD"][overlap], axis=1)[:, -2]

    # convert ids to boolean masks
    fjbbpick_mask = np.zeros_like(tvars["XbbvsQCD"], dtype=bool)
    fjbbpick_mask[np.arange(len(fjbbpick)), fjbbpick] = True
    fjttpick_mask = np.zeros_like(tvars[f"{taukey}vsQCD"], dtype=bool)
    fjttpick_mask[np.arange(len(fjttpick)), fjttpick] = True

    tvars["bb_mask"] = fjbbpick_mask
    tvars["tautau_mask"] = fjttpick_mask
    taggers_dict[key] = tvars

Checking bb matching accuracy (can skip)

In [ ]:
tvars = taggers_dict[SIG_KEY]
maxtxbb = np.max(tvars["XbbvsQCD"], axis=1)
fjbbpick = np.argmax(tvars["XbbvsQCD"], axis=1)
maxtxtt = np.max(tvars[f"{taukey}vsQCD"], axis=1)
fjttpick = np.argmax(tvars[f"{taukey}vsQCD"], axis=1)

# how many are assigned correctly?
print(np.mean(fjbbpick == fjidbb))  # 89.4%
print(np.mean(fjttpick == fjidtt))  # 91.2%

overlap = fjbbpick == fjttpick
print(np.mean(overlap))  # 21.1%
# how many pass reasonable tagger cuts?
print(np.sum((maxtxbb > 0.8) * (maxtxtt > 0.95) * overlap) / np.sum(overlap))  # <0.1%

In [25]:
def get_jet_vals(vals, mask):
    # check if vals is a numpy array
    if not isinstance(vals, np.ndarray):
        vals = vals.to_numpy()

    return vals[mask]

### ROC Curves

In [ ]:
from sklearn.metrics import roc_curve

rocs = {}

for jet in ["bb", "tautau"]:
    print(jet)
    rocs[jet] = {}
    for i, disc in enumerate(
        ["XbbvsQCD", "XbbvsQCDTop", f"{taukey}vsQCD", f"{taukey}vsQCDTop", "PNetXbbvsQCD"]
    ):
        print("\t" + disc)
        bg_scores = np.concatenate(
            [
                get_jet_vals(taggers_dict[key][disc], taggers_dict[key][f"{jet}_mask"])
                for key in DATA_KEYS
            ]
        )
        bg_weights = np.concatenate([events_dict[key]["finalWeight"] for key in DATA_KEYS])

        sig_scores = get_jet_vals(taggers_dict[SIG_KEY][disc], taggers_dict[SIG_KEY][f"{jet}_mask"])
        sig_weights = events_dict[SIG_KEY]["finalWeight"]

        fpr, tpr, thresholds = roc_curve(
            np.concatenate([np.zeros_like(bg_scores), np.ones_like(sig_scores)]),
            np.concatenate([bg_scores, sig_scores]),
            sample_weight=np.concatenate([bg_weights, sig_weights]),
        )

        rocs[jet][disc] = {
            "fpr": fpr,
            "tpr": tpr,
            "thresholds": thresholds,
            "label": disc,
            "color": plt.cm.tab10.colors[i],
        }

In [ ]:
for jet, title in zip(["bb", "tautau"], ["bb FatJet", rf"$\tau_h\tau_{CHANNEL[0]}$ FatJet"]):
    plotting.multiROCCurveGrey(
        {"": rocs[jet]}, title=title, show=True, plot_dir=plot_dir, name=f"roc_{jet}"
    )

## Mass

In [ ]:
for key, label in zip(["hhbbtt", "data"], ["HHbbtt", "Data"]):
    if key == "hhbbtt":
        events = events_dict[SIG_KEY]
    else:
        events = pd.concat([events_dict[dkey] for dkey in DATA_KEYS])

    bins = np.linspace(0, 250, 50)

    fig, axs = plt.subplots(1, 2, figsize=(24, 10))

    for i, (jet, jlabel) in enumerate(zip(["bb", "tautau"], ["bb FatJet", r"$\tau\tau$ FatJet"])):
        ax = axs[i]
        if key == "hhbbtt":
            mask = taggers_dict[SIG_KEY][f"{jet}_mask"]
        else:
            mask = np.concatenate([taggers_dict[dkey][f"{jet}_mask"] for dkey in DATA_KEYS], axis=0)

        for j, (mkey, mlabel) in enumerate(
            zip(
                [
                    "ak8FatJetMsd",
                    "ak8FatJetPNetmassLegacy",
                    "ak8FatJetParTmassResApplied",
                    "ak8FatJetParTmassVisApplied",
                ],
                ["SoftDrop", "PNetLegacy", "ParT Res", "ParT Vis"],
            )
        ):
            ax.hist(
                get_jet_vals(events[mkey], mask),
                bins=bins,
                histtype="step",
                weights=events["finalWeight"],
                label=mlabel,
                linewidth=2,
                color=plt.cm.tab10.colors[j],
            )

        ax.vlines(125, 0, ax.get_ylim()[1], linestyle="--", color="k", alpha=0.1)
        ax.set_title(jlabel, fontsize=24)
        ax.set_xlabel("Mass [GeV]")
        # rax.set_xlabel("Mass [GeV]")
        ax.set_ylabel("Events")
        ax.legend()
        ax.set_ylim(0)
        hep.cms.label(
            ax=ax,
            data=key == "data",
            year=2022,
            com="13.6",
            fontsize=20,
            lumi=f"{hh_vars.LUMI[year] / 1000:.1f}",
        )

    plt.savefig(plot_dir / f"{key}_mass.pdf", bbox_inches="tight")
    plt.show()

## Cut-and-count

In [ ]:
bbeff, tteff = 0.44, 0.36
mbb1, mbb2 = 110.0, 160.0
mbbw2 = (mbb2 - mbb1) / 2
mtt1, mtt2 = 50, 1500

# mbbk = "PNetmassLegacy"
mbbk = "ParTmassResApplied"
# mttk = "PNetmassLegacy"
mttk = "ParTmassResApplied"

txbbcut = rocs["bb"]["XbbvsQCD"]["thresholds"][
    plotting._find_nearest(rocs["bb"]["XbbvsQCD"]["tpr"], bbeff)
]
txttcut = rocs["tautau"][f"{taukey}vsQCDTop"]["thresholds"][
    plotting._find_nearest(rocs["tautau"][f"{taukey}vsQCDTop"]["tpr"], tteff)
]
print("TXbb cut, TXtt cut:", txbbcut, txttcut)

bg_yield = 0
sig_yield = 0

for key in [SIG_KEY] + DATA_KEYS:
    txbbs = get_jet_vals(taggers_dict[key]["XbbvsQCD"], taggers_dict[key]["bb_mask"])
    txtts = get_jet_vals(taggers_dict[key][f"{taukey}vsQCDTop"], taggers_dict[key]["tautau_mask"])
    masstt = get_jet_vals(events_dict[key][f"ak8FatJet{mttk}"], taggers_dict[key]["tautau_mask"])
    massbb = get_jet_vals(events_dict[key][f"ak8FatJet{mbbk}"], taggers_dict[key]["bb_mask"])
    ptbb = get_jet_vals(events_dict[key]["ak8FatJetPt"], taggers_dict[key]["bb_mask"])
    # plt.hist(massbb, np.linspace(0, 200, 100), histtype="step", label=key, weights=events_dict[key]["finalWeight"])

    if key == SIG_KEY:
        cut = (
            (txbbs > txbbcut)
            & (txtts > txttcut)
            & (masstt > mtt1)
            & (masstt < mtt2)
            & (massbb > mbb1)
            & (massbb < mbb2)
            & (ptbb > 250)
        )
        sig_yield = np.sum(events_dict[key]["finalWeight"][cut])
        print("Sig yield", sig_yield)
    else:
        cut = (
            (txbbs > txbbcut) & (txtts > txttcut) & (masstt > mtt1) & (masstt < mtt2) & (ptbb > 250)
        )
        msb1 = (massbb > (mbb1 - mbbw2)) & (massbb < mbb1)
        msb2 = (massbb > mbb2) & (massbb < (mbb2 + mbbw2))
        bg_yield += np.sum(events_dict[key]["finalWeight"][cut & msb1])
        bg_yield += np.sum(events_dict[key]["finalWeight"][cut & msb2])

        # bkg_yield = np.sum(events_dict[key]["finalWeight"][cut])
        # print("Bkg yield", bkg_yield)

# plt.yscale("log")
# plt.show()

print("BG yield", bg_yield)
print("limit", 2 * np.sqrt(bg_yield) / sig_yield)
print(
    "limit scaled to 22-23 all channels",
    2 * np.sqrt(bg_yield) / sig_yield / np.sqrt(hh_vars.LUMI["2022-2023"] / hh_vars.LUMI[year] * 3),
)
print(
    "limit scaled to 22-24 all channels",
    2
    * np.sqrt(bg_yield)
    / sig_yield
    / np.sqrt((124000 + hh_vars.LUMI["2022-2023"]) / hh_vars.LUMI[year] * 3),
)
print(
    "limit scaled to Run 3 all channels",
    2 * np.sqrt(bg_yield) / sig_yield / np.sqrt((360000) / hh_vars.LUMI[year] * 3),
)